# Transit Ridership Dashboard GTFS Refactor

- Migrating the transit ridership dashboard created Fall 2022 to warehouse v2

In [1]:
pip install shared_utils

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)

import branca
import folium
from shared_utils import gtfs_utils_v2

from siuba import *
import pandas as pd
import geopandas as gpd 

import datetime as dt
import time

In [3]:
# Creating function for datacheck
def analyze_dataset(df):
    #Number of rows and columns
    num_rows, num_cols = df.shape 
    print(f"Number of rows: {num_rows}, Number of columns: {num_cols}")
    print()
    
    # Print column names 
    column_names = df.columns.tolist()
    print(f"Column names: \n{column_names}\n")
    
    #Print data type
    print("Data type:")
    print(type(df))
    print()
          
    # Print data types
    print("Data types:")
    print(df.dtypes)
    print()
          
    # Check for duplicates
    duplicate_rows = df[df.duplicated()]
    if not duplicate_rows.empty:
          print("Duplicate rows:")
          print(duplicate_rows)
          print()
    else:
        print("No duplicate rows found \n")
            
    # Print first 3 words 
    print("First 3 rows:")
    display(df.head(3))
    print()

    

## Creating trips per weekday, saturday and sunday by stop

In [4]:
# using the 2022 data used in the previous dashboard.
analysis_dt = dt.date(2022,6,1)
analysis_sat = dt.date(2022,6,4)
analysis_sun = dt.date(2022,6,5)

analysis_operator_list = [182,293,208]

### Extracting Feed Data for Weekdays 

In [39]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_sat)

In [40]:
filtered_feeds = feeds[feeds['name'].str.contains('MTD')]

In [41]:
filtered_feeds

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,type,regional_feed_type
196,07cf74ca10f10d0c504828ccace9b5a9,2022-06-04,339cdaba8e9729e2b21b45754b842646,America/Los_Angeles,aHR0cDovL3NibXRkLmdvdi9nb29nbGVfdHJhbnNpdC9mZW...,fa0149a8b4dcc0fecc412387f0230fa7,SBMTD Schedule,schedule,None


### Selecting specific agencies : LA Metro

In [7]:
def select_by_agency(df, column, value):
    selected_df = df[df[column].str.contains(value)].copy()
    return selected_df

In [9]:
metrofeeds = select_by_agency(feeds, 'name', 'LA Metro')

In [10]:
analyze_dataset(metrofeeds)

Number of rows: 2, Number of columns: 9

Column names: 
['key', 'date', 'feed_key', 'feed_timezone', 'base64_url', 'gtfs_dataset_key', 'name', 'type', 'regional_feed_type']

Data type:
<class 'pandas.core.frame.DataFrame'>

Data types:
key                           object
date                  datetime64[ns]
feed_key                      object
feed_timezone                 object
base64_url                    object
gtfs_dataset_key              object
name                          object
type                          object
regional_feed_type            object
dtype: object

No duplicate rows found 

First 3 rows:


,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,name,type,regional_feed_type
84,288c891cdb625de4271a52c9f3e6daeb,2022-06-01,bc633d97886566eba81d46f81b0573b6,America/Los_Angeles,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaW...,683682f3c501f1edd5954f0a1f2a4d12,LA Metro Rail Schedule,schedule,None
131,b3d63accb016d1d47cb1ccdd73736206,2022-06-01,06d1f3ac2b0ae5e74424edbbfefa19ed,America/Los_Angeles,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,a09d454d421c1ef01e77b9e94aad0f5e,LA Metro Bus Schedule,schedule,None


In [9]:
# Feed key of LA metro in list format
metrofeed_list = metrofeeds.feed_key.to_list()

### Getting Trip Data for LA Metro for Weekdays

In [10]:
metro_trips = gtfs_utils_v2.get_trips(selected_date=analysis_dt, operator_feeds=metrofeed_list)

In [11]:
trip_cols = ["name", "gtfs_dataset_key", "feed_key",
             "trip_id", "route_id", "route_type"]

In [12]:
metro_trips = metro_trips[trip_cols]

In [13]:
analyze_dataset(metro_trips)

Number of rows: 13834, Number of columns: 6

Column names: 
['name', 'gtfs_dataset_key', 'feed_key', 'trip_id', 'route_id', 'route_type']

Data type:
<class 'pandas.core.frame.DataFrame'>

Data types:
name                object
gtfs_dataset_key    object
feed_key            object
trip_id             object
route_id            object
route_type          object
dtype: object

No duplicate rows found 

First 3 rows:


,name,gtfs_dataset_key,feed_key,trip_id,route_id,route_type
0,LA Metro Bus Schedule,a09d454d421c1ef01e77b9e94aad0f5e,06d1f3ac2b0ae5e74424edbbfefa19ed,DSE-HG-1650-DS-008,DSE-HG,3
1,LA Metro Bus Schedule,a09d454d421c1ef01e77b9e94aad0f5e,06d1f3ac2b0ae5e74424edbbfefa19ed,DSE-HG-1650-DS-007,DSE-HG,3
2,LA Metro Bus Schedule,a09d454d421c1ef01e77b9e94aad0f5e,06d1f3ac2b0ae5e74424edbbfefa19ed,DSE-HG-1650-DS-005,DSE-HG,3


### Getting Stop Times Data for Weekdays

In [14]:
metro_stops = gtfs_utils_v2.get_stop_times(selected_date=analysis_dt, operator_feeds=metrofeed_list, 
                                           trip_df = metro_trips, get_df= True)

/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'arrival_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'departure_time_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'start_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(
/opt/conda/lib/python3.9/site-packages/sqlalchemy_bigquery/_types.py:101: SAWarning: Did not recognize type 'INTERVAL' of column 'end_pickup_drop_off_window_interval'
  sqlalchemy.util.warn(


In [15]:
stop_cols=["key", "_gtfs_key", "feed_key", "trip_id", "stop_id"]

In [16]:
metro_stops= metro_stops[stop_cols]

In [17]:
analyze_dataset(metro_stops)

Number of rows: 793697, Number of columns: 5

Column names: 
['key', '_gtfs_key', 'feed_key', 'trip_id', 'stop_id']

Data type:
<class 'pandas.core.frame.DataFrame'>

Data types:
key          object
_gtfs_key    object
feed_key     object
trip_id      object
stop_id      object
dtype: object

No duplicate rows found 

First 3 rows:


,key,_gtfs_key,feed_key,trip_id,stop_id
0,bd8866407ade3e81eddbbe7ebe6e6e86,8bda692f5f1c6b0fd99b05990845b189,bc633d97886566eba81d46f81b0573b6,56064321,80109
1,1449dbb7bbac7b5e9ae8356de963b096,6a13fb05e30ae71fa772fdfb445c11d6,bc633d97886566eba81d46f81b0573b6,55217353,80427
2,7add6f0ae1867b4c6a23e0c32fd3e4ce,881d52b8a1aa430179c64b3d52155d74,bc633d97886566eba81d46f81b0573b6,55217353,80426


### Joining Stop Times and Trip Data 

In [18]:
metro_joined = pd.merge(
    metro_stops, metro_trips,
    on = ["trip_id", "feed_key"],
    how = 'left'
)                    

In [19]:
metro_joined.head(3)

,key,_gtfs_key,feed_key,trip_id,stop_id,name,gtfs_dataset_key,route_id,route_type
0,bd8866407ade3e81eddbbe7ebe6e6e86,8bda692f5f1c6b0fd99b05990845b189,bc633d97886566eba81d46f81b0573b6,56064321,80109,LA Metro Rail Schedule,683682f3c501f1edd5954f0a1f2a4d12,801,0
1,1449dbb7bbac7b5e9ae8356de963b096,6a13fb05e30ae71fa772fdfb445c11d6,bc633d97886566eba81d46f81b0573b6,55217353,80427,LA Metro Rail Schedule,683682f3c501f1edd5954f0a1f2a4d12,804,0
2,7add6f0ae1867b4c6a23e0c32fd3e4ce,881d52b8a1aa430179c64b3d52155d74,bc633d97886566eba81d46f81b0573b6,55217353,80426,LA Metro Rail Schedule,683682f3c501f1edd5954f0a1f2a4d12,804,0


### Finding Number of Trips on weekday per Stops

In [21]:
metrotrips_weekday =  metro_joined.groupby(['route_type', 'stop_id']).agg(
    ntrips_route = ('route_id', 'nunique'),
    ntrips_trip = ('trip_id', 'nunique')).reset_index()

In [28]:
metrotrips_weekday

,route_type,stop_id,ntrips_weekday,ntrips_route
0,0,80101,197,1
1,0,80102,94,1
2,0,80105,197,1
3,0,80106,197,1
4,0,80107,197,1
...,...,...,...,...
12246,3,9992,99,1
12247,3,9993,99,1
12248,3,9994,99,1
12249,3,9996,99,1


### Finding Number of Trips on Saturday per Stops

In [12]:
gtfs_utils_v2.get_stops?

Signature:
gtfs_utils_v2.get_stops(
    selected_date: Union[str, datetime.date],
    operator_feeds: list[str] = [],
    stop_cols: list[str] = [],
    get_df: bool = True,
    crs: str = 'EPSG:4326',
    custom_filtering: dict = None,
) -> Union[geopandas.geodataframe.GeoDataFrame, siuba.sql.backend.LazyTbl]
Docstring:
Query fct_daily_scheduled_stops.

Must supply a list of feed_keys or organization names returned from
schedule_daily_feed_to_gtfs_dataset_name() or subset of those results.
File:      ~/data-analyses/_shared_utils/shared_utils/gtfs_utils_v2.py
Type:      function

In [13]:
metro_st = shared_utils.gtfs_utils_v2.get_stops(selected_date=analysis_dt, operator_feeds=feed_list,
                                                    trip_df=metro_trips, get_df=True)

NameError: name 'shared_utils' is not defined